In [2]:
import sympy
from sympy import Matrix
from sympy import symbols
from sympy import simplify, factor
from sympy.solvers import solve

In [3]:
# Used in both
xTP, xFP, xTN, xFN = symbols('xTP xFP xTN xFN')
n = xTP + xFP + xTN + xFN

# Precision Recall

In [4]:
P, R = symbols('P R')

In [5]:
P = xTP / (xTP + xFP)
R = xTP / (xTP + xFN)
P

xTP/(xFP + xTP)

In [6]:
X = Matrix([P, R])
Y = Matrix([xTP, xFP, xTN, xFN])
J = X.jacobian(Y)
J

Matrix([
[-xTP/(xFP + xTP)**2 + 1/(xFP + xTP), -xTP/(xFP + xTP)**2, 0,                   0],
[-xTP/(xFN + xTP)**2 + 1/(xFN + xTP),                   0, 0, -xTP/(xFN + xTP)**2]])

In [7]:
# Used in both
covar_binomial = - (Y/n) * (Y/n).T
for i, x in enumerate(Y):
    covar_binomial[i,i] = x/n * (1-x/n)
covar_binomial = n* covar_binomial
covar_binomial

Matrix([
[xTP*(-xTP/(xFN + xFP + xTN + xTP) + 1),       -xFP*xTP/(xFN + xFP + xTN + xTP),       -xTN*xTP/(xFN + xFP + xTN + xTP),       -xFN*xTP/(xFN + xFP + xTN + xTP)],
[      -xFP*xTP/(xFN + xFP + xTN + xTP), xFP*(-xFP/(xFN + xFP + xTN + xTP) + 1),       -xFP*xTN/(xFN + xFP + xTN + xTP),       -xFN*xFP/(xFN + xFP + xTN + xTP)],
[      -xTN*xTP/(xFN + xFP + xTN + xTP),       -xFP*xTN/(xFN + xFP + xTN + xTP), xTN*(-xTN/(xFN + xFP + xTN + xTP) + 1),       -xFN*xTN/(xFN + xFP + xTN + xTP)],
[      -xFN*xTP/(xFN + xFP + xTN + xTP),       -xFN*xFP/(xFN + xFP + xTN + xTP),       -xFN*xTN/(xFN + xFP + xTN + xTP), xFN*(-xFN/(xFN + xFP + xTN + xTP) + 1)]])

In [8]:
covar_PR = J*covar_binomial*J.T
covar_PR = factor(covar_PR, deep=True)
covar_PR

Matrix([
[                     xFP*xTP/(xFP + xTP)**3, xFN*xFP*xTP/((xFN + xTP)**2*(xFP + xTP)**2)],
[xFN*xFP*xTP/((xFN + xTP)**2*(xFP + xTP)**2),                      xFN*xTP/(xFN + xTP)**3]])

# ROC

In [9]:
TPR, FPR = symbols('TPR FPR')

In [10]:
TPR = xTP / (xTP + xFN)  # Same as Recall: R
FPR = xFP / (xFP + xTN)
#FPR = xFP / (n - xTP - xFN)  # Same results

In [11]:
R == TPR

True

In [12]:
X_ROC = Matrix([TPR, FPR])
Y = Matrix([xTP, xFP, xTN, xFN])
J_ROC = X_ROC.jacobian(Y)
J_ROC

Matrix([
[-xTP/(xFN + xTP)**2 + 1/(xFN + xTP),                                   0,                   0, -xTP/(xFN + xTP)**2],
[                                  0, -xFP/(xFP + xTN)**2 + 1/(xFP + xTN), -xFP/(xFP + xTN)**2,                   0]])

In [13]:
covar_ROC = J_ROC * covar_binomial * J_ROC.T
covar_ROC = factor(covar_ROC, deep=True)
covar_ROC

Matrix([
[xFN*xTP/(xFN + xTP)**3,                      0],
[                     0, xFP*xTN/(xFP + xTN)**3]])

# Binomial Wilks

In [14]:
factor( (1-P)/P )

xFP/xTP

In [15]:
factor( (1-R)/R )

xFN/xTP

In [16]:
factor( (1-TPR)/TPR )

xFN/xTP

In [17]:
factor( (1-FPR)/FPR )

xTN/xFP

In [18]:
factor( FPR/(1-FPR) ) # Reverse

xFP/xTN

In [19]:
FPR/(1-FPR)

xFP/((xFP + xTN)*(-xFP/(xFP + xTN) + 1))

In [20]:
factor( (FPR/(1-FPR)) * xTN )

xFP

In [21]:
factor( (FPR/(1-FPR)) * (1-xTP-xFP-xFN) )

-xFP*(xFN + xFP + xTP - 1)/xTN

In [22]:
factor( (FPR/(1-FPR)) * (1-xTP-xFP-((1-TPR)/TPR * xTP)) )

-xFP*(xFN + xFP + xTP - 1)/xTN

## Which to proba

In [23]:
# Which to probability
pTP, pFP, pTN, pFN = symbols('pTP pFP pTN pFN', real=True)
#1 = pTP + pFP + pTN + pFN

In [24]:
# Constant
cP, cR, cFPR, cTPR, cN = symbols('cP cR cFPR cTPR cN')

In [25]:
print("pFP = ")
solve(pFP - ( (cFPR/(1-cFPR)) * (1-pTP-pFP-pFN) ), pFP)[0]

pFP = 


cFPR*(-pFN - pTP + 1)

In [26]:
print("pFP = ")
factor(solve(pFP - ( (cFPR/(1-cFPR)) * (1-pTP-pFP- ( ((1-cTPR)/cTPR)*pTP ) ) ), pFP)[0])

pFP = 


cFPR*(cTPR - pTP)/cTPR

## Analytical solution for ROC

In [27]:
pFP = (cFPR)/(1-cFPR) * pTN

In [28]:
pFN = (1-cTPR)/cTPR * pTP

In [29]:
pTN = 1 - pTP - pFP - pFN

In [30]:
pFP = (cFPR)/(1-cFPR) * pTN
pFP = (cFPR)/(1-cFPR) * (1 - pTP - pFP - pFN)
pFP = solve((cFPR)/(1-cFPR) * (1 - pTP - pFP - pFN) - pFP, pFP)[0]
pFP

cFPR*(cTPR - pTP)/cTPR

In [31]:
proba = pFP**xFP * pFN**xFN * pTP**xTP * (1 - pTP - pFP - pFN)**xTN
proba

pTP**xTP*(cFPR*(cTPR - pTP)/cTPR)**xFP*(pTP*(1 - cTPR)/cTPR)**xFN*(-cFPR*(cTPR - pTP)/cTPR - pTP + 1 - pTP*(1 - cTPR)/cTPR)**xTN

In [32]:
proba_log = sympy.log(proba)
proba_log

log(pTP**xTP*(cFPR*(cTPR - pTP)/cTPR)**xFP*(pTP*(1 - cTPR)/cTPR)**xFN*(-cFPR*(cTPR - pTP)/cTPR - pTP + 1 - pTP*(1 - cTPR)/cTPR)**xTN)

In [33]:
log_proba_diff = sympy.diff(proba_log, pTP)
log_proba_diff

(-pTP**xTP*xFP*(cFPR*(cTPR - pTP)/cTPR)**xFP*(pTP*(1 - cTPR)/cTPR)**xFN*(-cFPR*(cTPR - pTP)/cTPR - pTP + 1 - pTP*(1 - cTPR)/cTPR)**xTN/(cTPR - pTP) + pTP**xTP*xTN*(cFPR*(cTPR - pTP)/cTPR)**xFP*(pTP*(1 - cTPR)/cTPR)**xFN*(cFPR/cTPR - 1 - (1 - cTPR)/cTPR)*(-cFPR*(cTPR - pTP)/cTPR - pTP + 1 - pTP*(1 - cTPR)/cTPR)**xTN/(-cFPR*(cTPR - pTP)/cTPR - pTP + 1 - pTP*(1 - cTPR)/cTPR) + pTP**xTP*xFN*(cFPR*(cTPR - pTP)/cTPR)**xFP*(pTP*(1 - cTPR)/cTPR)**xFN*(-cFPR*(cTPR - pTP)/cTPR - pTP + 1 - pTP*(1 - cTPR)/cTPR)**xTN/pTP + pTP**xTP*xTP*(cFPR*(cTPR - pTP)/cTPR)**xFP*(pTP*(1 - cTPR)/cTPR)**xFN*(-cFPR*(cTPR - pTP)/cTPR - pTP + 1 - pTP*(1 - cTPR)/cTPR)**xTN/pTP)/(pTP**xTP*(cFPR*(cTPR - pTP)/cTPR)**xFP*(pTP*(1 - cTPR)/cTPR)**xFN*(-cFPR*(cTPR - pTP)/cTPR - pTP + 1 - pTP*(1 - cTPR)/cTPR)**xTN)

In [35]:
factor(log_proba_diff)

(cTPR*xFN + cTPR*xTP - pTP*xFN - pTP*xFP - pTP*xTN - pTP*xTP)/(pTP*(cTPR - pTP))

In [33]:
solutions = solve(log_proba_diff, pTP)
print("Number of solution:", len(solutions))
solutions[-1]

Number of solution: 1


cTPR*(xFN + xTP)/(xFN + xFP + xTN + xTP)

In [ ]:
# TPR = xTP / (xTP + xFN)  # Same as Recall: R

In [34]:
simplify(factor(solutions[-1]))

cTPR*(xFN + xTP)/(xFN + xFP + xTN + xTP)

In [35]:
sympy.print_python(simplify(factor(solutions[-1])))

cTPR = Symbol('cTPR')
xFN = Symbol('xFN')
xTP = Symbol('xTP')
xFP = Symbol('xFP')
xTN = Symbol('xTN')
e = cTPR*(xFN + xTP)/(xFN + xFP + xTN + xTP)


In [36]:
1/0

ZeroDivisionError: division by zero

## Analytical solution for PR

In [37]:
pFP = (1-cP)/cP * pTP
pFP

pTP*(1 - cP)/cP

In [38]:
pFN = (1-cR)/cR * pTP
pFN

pTP*(1 - cR)/cR

In [39]:
pFP**xFP * pFN**xFN * pTP**xTP * pTN**xTN

pTP**xTP*(pTP*(1 - cP)/cP)**xFP*(pTP*(1 - cR)/cR)**xFN*(-cFPR*pTN/(1 - cFPR) - pTP + 1 - pTP*(1 - cTPR)/cTPR)**xTN

In [40]:
xTN = cN - xTP - xFP - xFN
#xTP = cN - xTN - xFP - xFN

In [41]:
proba = pFP**xFP * pFN**xFN * pTP**xTP * (1-pFP-pFN-pTP)**xTN
proba

pTP**xTP*(pTP*(1 - cP)/cP)**xFP*(pTP*(1 - cR)/cR)**xFN*(-pTP + 1 - pTP*(1 - cR)/cR - pTP*(1 - cP)/cP)**(cN - xFN - xFP - xTP)

In [42]:
proba_log = sympy.log(proba)
proba_log

log(pTP**xTP*(pTP*(1 - cP)/cP)**xFP*(pTP*(1 - cR)/cR)**xFN*(-pTP + 1 - pTP*(1 - cR)/cR - pTP*(1 - cP)/cP)**(cN - xFN - xFP - xTP))

In [43]:
sympy.log(proba).expand(force=True)

cN*log(pTP + 1 - pTP/cR - pTP/cP) - xFN*log(cR) + xFN*log(pTP) + xFN*log(1 - cR) - xFN*log(pTP + 1 - pTP/cR - pTP/cP) - xFP*log(cP) + xFP*log(pTP) + xFP*log(1 - cP) - xFP*log(pTP + 1 - pTP/cR - pTP/cP) + xTP*log(pTP) - xTP*log(pTP + 1 - pTP/cR - pTP/cP)

In [44]:
simplify(sympy.log(proba), inverse=True)

log(pTP**xTP*(pTP*(1 - cP)/cP)**xFP*(pTP*(1 - cR)/cR)**xFN*(-pTP + 1 - pTP*(1 - cR)/cR - pTP*(1 - cP)/cP)**(cN - xFN - xFP - xTP))

In [45]:
sympy.print_python(proba)

xTP = Symbol('xTP')
pTP = Symbol('pTP')
xFP = Symbol('xFP')
cP = Symbol('cP')
xFN = Symbol('xFN')
cR = Symbol('cR')
cN = Symbol('cN')
e = pTP**xTP*(pTP*(1 - cP)/cP)**xFP*(pTP*(1 - cR)/cR)**xFN*(-pTP + 1 - pTP*(1 - cR)/cR - pTP*(1 - cP)/cP)**(cN - xFN - xFP - xTP)


In [46]:
log_proba_diff = sympy.diff(proba_log, pTP)
log_proba_diff

(-pTP + 1 - pTP*(1 - cR)/cR - pTP*(1 - cP)/cP)**(-cN + xFN + xFP + xTP)*(pTP**xTP*(pTP*(1 - cP)/cP)**xFP*(pTP*(1 - cR)/cR)**xFN*(-1 - (1 - cR)/cR - (1 - cP)/cP)*(cN - xFN - xFP - xTP)*(-pTP + 1 - pTP*(1 - cR)/cR - pTP*(1 - cP)/cP)**(cN - xFN - xFP - xTP)/(-pTP + 1 - pTP*(1 - cR)/cR - pTP*(1 - cP)/cP) + pTP**xTP*xFN*(pTP*(1 - cP)/cP)**xFP*(pTP*(1 - cR)/cR)**xFN*(-pTP + 1 - pTP*(1 - cR)/cR - pTP*(1 - cP)/cP)**(cN - xFN - xFP - xTP)/pTP + pTP**xTP*xFP*(pTP*(1 - cP)/cP)**xFP*(pTP*(1 - cR)/cR)**xFN*(-pTP + 1 - pTP*(1 - cR)/cR - pTP*(1 - cP)/cP)**(cN - xFN - xFP - xTP)/pTP + pTP**xTP*xTP*(pTP*(1 - cP)/cP)**xFP*(pTP*(1 - cR)/cR)**xFN*(-pTP + 1 - pTP*(1 - cR)/cR - pTP*(1 - cP)/cP)**(cN - xFN - xFP - xTP)/pTP)/(pTP**xTP*(pTP*(1 - cP)/cP)**xFP*(pTP*(1 - cR)/cR)**xFN)

In [47]:
factor(log_proba_diff)

(pTP + 1 - pTP/cR - pTP/cP)**(-cN + xFN + xFP + xTP)*(pTP + 1 - pTP/cR - pTP/cP)**(cN - xFN - xFP - xTP)*(cN*cP*cR*pTP - cN*cP*pTP - cN*cR*pTP + cP*cR*xFN + cP*cR*xFP + cP*cR*xTP)/(pTP*(cP*cR*pTP + cP*cR - cP*pTP - cR*pTP))

In [48]:
log_proba_diff = simplify(factor(log_proba_diff))
log_proba_diff

(cN*cP*cR*pTP - cN*cP*pTP - cN*cR*pTP + cP*cR*xFN + cP*cR*xFP + cP*cR*xTP)/(pTP*(cP*cR*pTP + cP*cR - cP*pTP - cR*pTP))

In [49]:
solutions = solve(log_proba_diff, pTP)
print("Number of solution:", len(solutions))
solutions[-1]

Number of solution: 1


cP*cR*(xFN + xFP + xTP)/(cN*(-cP*cR + cP + cR))

In [50]:
solutions[0]

cP*cR*(xFN + xFP + xTP)/(cN*(-cP*cR + cP + cR))